
## credit score classifier

[Data Source](https://www.kaggle.com/datasets/parisrohan/credit-score-classification)


Credit Scores will have one of three values:
* `Good`
* `Standard`
* `Poor`

##### Data Notes

The same users appear multiple times in both the train and test sets. 

In my opinion, this train/test split is not a great way to measure model performance
The credit score of a single person in two different months are going to be highly correlated,
and could be preceived as a form of data leakage.

A more reasonble training method would be to only have one row for each person,
such that a person can only appear in train or test.

In [6]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)
%config Completer.use_jedi = False

Read in Data

In [7]:

# since the test.csv doesn't have labels, we can't use it to evaluate model performance
credit_df = pd.read_csv("train.csv", low_memory=False)
credit_df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",23,7,11.5,3.0,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.86622816574095,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",18,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.651610435322,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,5729,2,"Auto Loan, and Student Loan",27,6,11.5,3.0,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.8090832742814,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",20,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.1649785257098,Standard


Data Cleaning

In [8]:
# ---- Age ----
credit_df["Age"] = pd.to_numeric(credit_df["Age"], errors="coerce")
bad_ages = (credit_df["Age"] < 0) | (credit_df["Age"] > 120)
credit_df.loc[bad_ages, ["Age"]] = np.nan

# ---- Occupation ---- 
occ_one_hot_df = pd.get_dummies(credit_df["Occupation"])
occ_one_hot_df.columns = ["Occupation_"+ c for c in occ_one_hot_df.columns]
credit_df = pd.concat([occ_one_hot_df, credit_df], axis=1)

In [9]:
# ---- Annual_Income ---- 
credit_df["Annual_Income"] = pd.to_numeric(credit_df["Annual_Income"], errors="coerce")

# ---- Amount_Invested_Monthly
credit_df["Amount_invested_monthly"] = pd.to_numeric(credit_df["Amount_invested_monthly"], errors="coerce")

# ---- Num_of_Loan ---- 
credit_df["Num_of_Loan"] = pd.to_numeric(credit_df["Num_of_Loan"], errors="coerce")

# ---- Num_of_Delayed_Payment ----
credit_df["Num_of_Delayed_Payment"] = pd.to_numeric(credit_df["Num_of_Delayed_Payment"], errors="coerce")

# ---- Changed_Credit_Limit ----
credit_df["Changed_Credit_Limit"] = pd.to_numeric(credit_df["Changed_Credit_Limit"], errors="coerce")

# ---- Outstanding_Debt ----
credit_df["Outstanding_Debt"] = pd.to_numeric(credit_df["Outstanding_Debt"], errors="coerce")

# ---- Credit_History_Age ----
credit_age_df = credit_df["Credit_History_Age"].str.split(' Years and ', expand=True).iloc[100:150]
credit_age_df.columns = ["Years", "Months"]
credit_age_df["Years"] = pd.to_numeric(credit_age_df["Years"])
credit_age_df["Months"] = pd.to_numeric(credit_age_df["Months"].str.split(" ", expand=True)[0])
credit_df["Credit_History_Age"] = credit_age_df["Years"] + credit_age_df["Months"]/12

# ---- Payment_of_Min_Amount ----
min_amount_one_hot_df = pd.get_dummies(credit_df["Payment_of_Min_Amount"])
min_amount_one_hot_df.columns = ["Payment_of_Min_Amount_"+ c for c in min_amount_one_hot_df.columns]
credit_df = pd.concat([min_amount_one_hot_df, credit_df], axis=1)


# ---- Monthly_Balance ----
credit_df["Monthly_Balance"] = pd.to_numeric(credit_df["Monthly_Balance"], errors="coerce")


# ---- Type of Loan --- 
# for string column replace nan with empty string
credit_df.loc[credit_df["Type_of_Loan"].isna(),["Type_of_Loan"]] = ""


# vectorize our list of loans by splitting on the "," delimiter
count_vectorizer = CountVectorizer(tokenizer=lambda text: re.split(",\s+|,|\s+,",text))
loan_types = count_vectorizer.fit_transform(credit_df["Type_of_Loan"])


# convert sparse array back into a dataframe
loan_types_df = pd.DataFrame.sparse.from_spmatrix(loan_types,
                                                  columns=count_vectorizer.get_feature_names_out())

loan_types_df.drop(columns="", inplace=True)
credit_df = pd.concat([loan_types_df, credit_df], axis=1)


# ---- Credit Mix ----
credit_mix_one_hot_df = pd.get_dummies(credit_df["Credit_Mix"])
credit_mix_one_hot_df.columns = ["Credit_Mix_"+ c for c in credit_mix_one_hot_df.columns]
credit_df = pd.concat([credit_mix_one_hot_df, credit_df], axis=1)


# ---- Payment_Behavior ----
pb_one_hot_df = pd.get_dummies(credit_df["Payment_Behaviour"])
pb_one_hot_df.columns = ["Payment_Behavior_"+ c for c in pb_one_hot_df.columns]
credit_df = pd.concat([pb_one_hot_df, credit_df], axis=1)

####### Our Target Variable ###### 
le = LabelEncoder()
credit_df["Credit_Score"] = le.fit_transform(credit_df["Credit_Score"])


# columns that identify a person, or have already been converted into a 
# numeric value
drop_cols = ["ID",  "Month", "Name", "SSN", "Occupation", 
             "Type_of_Loan", "Credit_Mix", "Payment_of_Min_Amount",
             "Payment_Behaviour"] # "Customer_ID",


credit_df.drop(drop_cols, axis=1, inplace=True)


# shift column 'Customer_ID' to first position
first_column = credit_df.pop('Customer_ID')
  
# insert column using insert(position,column_name,first_column) function
credit_df.insert(0, 'Customer_ID', first_column)
  

### Aggregation

Take the first non-null value for each person

In [11]:
credit_agg_df = credit_df.groupby('Customer_ID', as_index=False).first()
credit_agg_df

,Customer_ID,Payment_Behavior_!@9#%8,Payment_Behavior_High_spent_Large_value_payments,Payment_Behavior_High_spent_Medium_value_payments,Payment_Behavior_High_spent_Small_value_payments,Payment_Behavior_Low_spent_Large_value_payments,Payment_Behavior_Low_spent_Medium_value_payments,Payment_Behavior_Low_spent_Small_value_payments,Credit_Mix_Bad,Credit_Mix_Good,Credit_Mix_Standard,Credit_Mix__,and auto loan,and credit-builder loan,and debt consolidation loan,and home equity loan,and mortgage loan,and not specified,and payday loan,and personal loan,and student loan,auto loan,credit-builder loan,debt consolidation loan,home equity loan,mortgage loan,not specified,payday loan,personal loan,student loan,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Occupation________,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score
0,CUS_0x1000,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,17.0,30625.940,2706.161667,6,5,27,2.0,62,25.0,1.63,10.0,1562.91,26.612093,NaN,42.941090,244.750283,252.924793,2
1,CUS_0x1009,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,25.0,52312.680,4250.390000,6,5,17,4.0,8,1749.0,9.73,2.0,202.68,23.679534,NaN,108.366467,261.210979,335.461554,2
2,CUS_0x100b,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,18.0,113781.390,9549.782500,1,4,1,0.0,14,7.0,11.34,1.0,1030.20,37.998760,NaN,0.000000,397.241846,837.736404,1
3,CUS_0x1011,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,44.0,58918.470,5208.872500,3,3,17,3.0,27,12.0,14.42,7.0,473.14,27.513599,NaN,123.434939,74.567386,562.884925,2
4,CUS_0x1013,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,43.0,98620.980,7962.415000,3,3,6,3.0,12,9.0,1.33,3.0,1233.51,41.920614,NaN,228.018084,258.694588,559.528828,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0xff3,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,55.0,17032.785,1176.398750,0,6,2,3.0,9,9.0,12.86,3.0,1229.08,34.082750,NaN,33.299764,43.044062,291.296050,0
12496,CUS_0xff4,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,36.0,25546.260,2415.855000,8,7,14,5.0,16,14.0,7.83,5.0,758.44,37.418209,18.166667,101.328637,53.104599,337.152264,2
12497,CUS_0xff6,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,18.0,117639.920,9727.326667,5,6,1,2.0,1,4.0,10.40,2.0,338.30,42.501107,NaN,126.638453,361.418069,734.676144,2
12498,CUS_0xffc,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,2,0,1,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,17.0,60877.170,5218.097500,6,8,27,8.0,46,16.0,5.82,8.0,1300.13,36.010476,NaN,272.809169,305.078298,223.922283,1


#### Inspect for Nan

In [12]:
null_counts = credit_agg_df.isna().sum()
has_nulls = null_counts > 0

null_counts[has_nulls].sort_values()

Credit_History_Age    12493
dtype: int64

`Credit_History_Age` is the only remaining column with names.

Let's fill values with mean. 
If we want to get fancier, we could fill using knn instead.

In [13]:
mean_credit_age = credit_agg_df["Credit_History_Age"].mean()
credit_agg_df.loc[credit_agg_df["Credit_History_Age"].isna(), ["Credit_History_Age"]] = mean_credit_age

##### Make sure that all columns are some sort of numeric value

In [14]:
col_dtypes = credit_agg_df.dtypes 
col_dtypes[col_dtypes == "object"]

Customer_ID    object
dtype: object

#### Drop  Customer_ID col
We no longer need it

In [15]:
credit_agg_df.drop(columns=["Customer_ID"], inplace=True)

### Save Processed Data to CSV

In [16]:
credit_agg_df.to_csv("data_cleaned.csv", header=True, index=False)

#### Train / Dev / Test

Our data set isn't that big.
If we need more data, we could fold our dev set into our train set and use cross validation.


In [17]:
# first split to get train
credit_agg_train_df, credit_agg_rem_df = train_test_split(credit_agg_df,
                                                          test_size=0.3,
                                                          shuffle=True,
                                                          random_state=99)


# split again to get dev and test
credit_agg_dev_df, credit_agg_test_df = train_test_split(credit_agg_rem_df,
                                                          test_size=0.5,
                                                          shuffle=True,
                                                          random_state=99)


# print row counts to make sure that sampling was done correctly
print("train count: {}".format(credit_agg_train_df.shape))
print("dev count: {}".format(credit_agg_dev_df.shape))
print("test count: {}".format(credit_agg_test_df.shape))


# write to file
credit_agg_train_df.to_csv("train_cleaned.csv", header=True, index=False)
credit_agg_dev_df.to_csv("dev_cleaned.csv", header=True, index=False)
credit_agg_test_df.to_csv("test_cleaned.csv", header=True, index=False)

train count: (8750, 66)
dev count: (1875, 66)
test count: (1875, 66)
